In [1]:
# basic-module
# 参考にしたpytorch　tutorialsデータについて
# https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

import matplotlib.pyplot as plt
import sys,os,re,glob
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
import seaborn as sns

# deep learning modules 
# imports

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


DHOME="/work/ysorimachi/deepl/pytorch/PennFudanPed"

In [3]:
from PIL import Image

class PennFudanDataset(Dataset):
    def __init__(self,root,trainsforms):
        self.root = root
        self.transforms = transforms
        
        #list - datas
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))
        
    
    def __getitems__(self,idx):
        img_path = os.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.join(self.root, "PedMasks", self.masks[idx])
        
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path) #not convert RGB(for mask)
        
        obj_ids  = np.unique(mask)
        obj_ids = obj_ids[1:]
        
        masks = mask == obj_ids[:, None, None]
        #get_bounding box
        boxes = [] #4隅の情報を取得するプログラム
        for i in range(num_ids):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin,ymin,xmax,ymax])
        
        boxes = torch.as_tensor(boxes, dtype= torch.float32) #convert -> torch tensor mode
        labels = torch.ones((num_obj,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype= torch.unit8)
        
        image_id = torch.tensor([idx])
        area = (boxes[:,3]- boxes[:1]) *  (boxes[:,2]- boxes[:0]) #sample 数分の面積を格納しておくようなもの
        iscrowd = torch.zeros((num_obj,), dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = albels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
            img,target = self.transforms(img,target)
            
        return img,target
    
    def __len__(self):
        return len(self.imgs)   

In [5]:
#すでに学習済みのmodelを拝借して利用する
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

#
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
# 学習済のモデルについては、こちらで置き換えていく必要があるので、最初のheads部分のみ上書きしていくような感じになる
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /home/ysorimachi/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


In [6]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

In [8]:
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone

#ConvBNReLU の説明を調べたら、このサイトにとんだ
# https://tzmi.hatenablog.com/entry/2020/02/06/183314
# 面白い発見
# swish activation layer について　ReLU * Sigmoid 関数の形をしているような気がする


Sequential(
  (0): ConvBNReLU(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (1): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (2): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      